# **Amazon Lookout for Equipment** - Getting started
*Part 2 - Dataset creation*

## Initialization
---
This repository is structured as follow:

```sh
. lookout-equipment-demo
|
├── data/
|   ├── interim                          # Temporary intermediate data are stored here
|   ├── processed                        # Finalized datasets are usually stored here
|   |                                    # before they are sent to S3 to allow the
|   |                                    # service to reach them
|   └── raw                              # Immutable original data are stored here
|
├── getting_started/
|   ├── 1_data_preparation.ipynb
|   ├── 2_dataset_creation.ipynb         <<< THIS NOTEBOOK <<<
|   ├── 3_model_training.ipynb
|   ├── 4_model_evaluation.ipynb
|   ├── 5_inference_scheduling.ipynb
|   └── 6_cleanup.ipynb
|
└── utils/
    └── lookout_equipment_utils.py
```

### Notebook configuration update

In [2]:
!pip install --quiet --upgrade sagemaker tqdm lookoutequipment

### Imports

In [3]:
import boto3
import config
import os
import pandas as pd
import sagemaker
import sys
import time

from datetime import datetime

# SDK / toolbox for managing Lookout for Equipment API calls:
import lookoutequipment as lookout

In [4]:
PROCESSED_DATA = os.path.join('..', 'data', 'processed', 'getting-started')
TRAIN_DATA     = os.path.join(PROCESSED_DATA, 'training-data')

ROLE_ARN       = sagemaker.get_execution_role()
REGION_NAME    = boto3.session.Session().region_name
DATASET_NAME   = config.DATASET_NAME
BUCKET         = config.BUCKET
PREFIX         = config.PREFIX_TRAINING

## Create a dataset
---

In [5]:
lookout_dataset = lookout.LookoutEquipmentDataset(
    dataset_name=DATASET_NAME,
    component_root_dir=TRAIN_DATA,
    access_role_arn=ROLE_ARN
)

Let's double check the schema detected for our dataset to ensure we are pointing to the right data:

In [6]:
import pprint
pp = pprint.PrettyPrinter(depth=5)
pp.pprint(eval(lookout_dataset.dataset_schema))

{'Components': [{'Columns': [{'Name': 'Timestamp', 'Type': 'DATETIME'},
                             {'Name': 'Sensor0', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor1', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor2', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor3', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor4', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor5', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor6', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor7', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor8', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor9', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor10', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor11', 'Type': 'DOUBLE'},
                             {'Name': 'Sensor24', 'Type': 'DOUBLE'},
                             {'Name': 'Se

The following method encapsulate the [**CreateDataset**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_CreateDataset.html) API:

```python
lookout_client.create_dataset(
    DatasetName=self.dataset_name,
    DatasetSchema={
        'InlineDataSchema': "schema"
    }
)
```

In [8]:
lookout_dataset.create()

Dataset "getting-started-pump" does not exist, creating it...



{'DatasetName': 'getting-started-pump',
 'DatasetArn': 'arn:aws:lookoutequipment:eu-west-1:123031033346:dataset/getting-started-pump/5977e2f5-9ace-4602-a8cd-3f6f3aed9096',
 'Status': 'CREATED',
 'ResponseMetadata': {'RequestId': '11838471-94f7-46ef-89fd-f26b57b4a317',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11838471-94f7-46ef-89fd-f26b57b4a317',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '186',
   'date': 'Wed, 30 Mar 2022 08:14:41 GMT'},
  'RetryAttempts': 0}}

The dataset is now created, but it is empty and ready to receive some timeseries data that we will ingest from the S3 location prepared in the previous notebook:

![Dataset created](assets/dataset-created.png)

## Ingest data into a dataset
---
Let's double check the values of all the parameters that will be used to ingest some data into an existing Lookout for Equipment dataset:

In [10]:
ROLE_ARN, BUCKET, PREFIX, DATASET_NAME

('arn:aws:iam::123031033346:role/service-role/AmazonSageMaker-ExecutionRole-20200922T205159',
 'lookout-equipment-getting-started',
 'getting_started/training-data/',
 'getting-started-pump')

Launch the ingestion job in the Lookout for Equipment dataset: the following method encapsulates the [**StartDataIngestionJob**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_StartDataIngestionJob.html) API:

```python
lookout_client.start_data_ingestion_job(
    DatasetName=DATASET_NAME,
    RoleArn=ROLE_ARN, 
    IngestionInputConfiguration={ 
        'S3InputConfiguration': { 
            'Bucket': BUCKET,
            'Prefix': PREFIX
        }
    }
)
```

In [11]:
response = lookout_dataset.ingest_data(BUCKET, PREFIX)

The ingestion is launched. With this amount of data (around 50 MB), it should take between less than 5 minutes:

![dataset_schema](assets/dataset-ingestion-in-progress.png)

We use the following cell to monitor the ingestion process by calling the following method, which encapsulates the [**DescribeDataIngestionJob**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DescribeDataIngestionJob.html) API and runs it every 60 seconds:

In [12]:
lookout_dataset.poll_data_ingestion(sleep_time=60)

2022-03-30 08:18:59 | Data ingestion: IN_PROGRESS
2022-03-30 08:19:59 | Data ingestion: IN_PROGRESS
2022-03-30 08:20:59 | Data ingestion: SUCCESS


In case any issue arise, you can inspect the API response available as a JSON document:

In [13]:
lookout_dataset.ingestion_job_response

{'JobId': 'ccb31f90e1893aac76b989e1e4e948c2',
 'DatasetArn': 'arn:aws:lookoutequipment:eu-west-1:123031033346:dataset/getting-started-pump/5977e2f5-9ace-4602-a8cd-3f6f3aed9096',
 'IngestionInputConfiguration': {'S3InputConfiguration': {'Bucket': 'lookout-equipment-getting-started',
   'Prefix': 'getting_started/training-data/'}},
 'RoleArn': 'arn:aws:iam::123031033346:role/service-role/AmazonSageMaker-ExecutionRole-20200922T205159',
 'CreatedAt': datetime.datetime(2022, 3, 30, 8, 17, 7, 757000, tzinfo=tzlocal()),
 'Status': 'SUCCESS',
 'ResponseMetadata': {'RequestId': '1a035b2a-fab8-45df-ae4c-a4a48287beb9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1a035b2a-fab8-45df-ae4c-a4a48287beb9',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '1081',
   'date': 'Wed, 30 Mar 2022 08:20:59 GMT'},
  'RetryAttempts': 0}}

The ingestion should now be complete as can be seen in the console:

![Ingestion done](assets/dataset-ingestion-done.png)

## Conclusion
---

In this notebook, we created a **Lookout for Equipment dataset** and ingested the S3 data previously uploaded into this dataset. **Move now to the next notebook to train a model based on these data.**